# پیاده‌سازی یک سیستم خبره حیوانات

مثالی از [برنامه درسی هوش مصنوعی برای مبتدیان](http://github.com/microsoft/ai-for-beginners).

در این نمونه، ما یک سیستم ساده مبتنی بر دانش را پیاده‌سازی خواهیم کرد که بر اساس برخی ویژگی‌های فیزیکی، یک حیوان را شناسایی می‌کند. این سیستم می‌تواند با درخت AND-OR زیر نمایش داده شود (این فقط بخشی از کل درخت است و می‌توانیم به‌راحتی قوانین بیشتری اضافه کنیم):

![](../../../../lessons/2-Symbolic/images/AND-OR-Tree.png)


## پوسته سیستم‌های خبره خودمان با استنتاج معکوس

بیایید یک زبان ساده برای نمایش دانش بر اساس قوانین تولید تعریف کنیم. از کلاس‌های پایتون به عنوان کلمات کلیدی برای تعریف قوانین استفاده خواهیم کرد. اساساً سه نوع کلاس وجود خواهد داشت:
* `Ask` نمایانگر یک سوال است که باید از کاربر پرسیده شود. این کلاس شامل مجموعه‌ای از پاسخ‌های ممکن است.
* `If` نمایانگر یک قانون است و فقط یک شکر نحوی برای ذخیره محتوای قانون است.
* `AND`/`OR` کلاس‌هایی هستند که شاخه‌های AND/OR درخت را نمایش می‌دهند. این کلاس‌ها فقط لیست آرگومان‌ها را در داخل خود ذخیره می‌کنند. برای ساده‌سازی کد، تمام قابلیت‌ها در کلاس والد `Content` تعریف شده‌اند.


In [1]:
class Ask():
    def __init__(self,choices=['y','n']):
        self.choices = choices
    def ask(self):
        if max([len(x) for x in self.choices])>1:
            for i,x in enumerate(self.choices):
                print("{0}. {1}".format(i,x),flush=True)
            x = int(input())
            return self.choices[x]
        else:
            print("/".join(self.choices),flush=True)
            return input()

class Content():
    def __init__(self,x):
        self.x=x
        
class If(Content):
    pass

class AND(Content):
    pass

class OR(Content):
    pass

در سیستم ما، حافظه کاری شامل فهرستی از **حقایق** به صورت **جفت‌های ویژگی-مقدار** خواهد بود. پایگاه دانش می‌تواند به عنوان یک دیکشنری بزرگ تعریف شود که اقدامات (حقایق جدیدی که باید در حافظه کاری وارد شوند) را به شرایطی که به صورت عبارات AND-OR بیان شده‌اند، نگاشت می‌کند. همچنین، برخی حقایق می‌توانند `پرسیده شوند`.


In [2]:
rules = {
    'default': Ask(['y','n']),
    'color' : Ask(['red-brown','black and white','other']),
    'pattern' : Ask(['dark stripes','dark spots']),
    'mammal': If(OR(['hair','gives milk'])),
    'carnivor': If(OR([AND(['sharp teeth','claws','forward-looking eyes']),'eats meat'])),
    'ungulate': If(['mammal',OR(['has hooves','chews cud'])]),
    'bird': If(OR(['feathers',AND(['flies','lies eggs'])])),
    'animal:monkey' : If(['mammal','carnivor','color:red-brown','pattern:dark spots']),
    'animal:tiger' : If(['mammal','carnivor','color:red-brown','pattern:dark stripes']),
    'animal:giraffe' : If(['ungulate','long neck','long legs','pattern:dark spots']),
    'animal:zebra' : If(['ungulate','pattern:dark stripes']),
    'animal:ostrich' : If(['bird','long nech','color:black and white','cannot fly']),
    'animal:pinguin' : If(['bird','swims','color:black and white','cannot fly']),
    'animal:albatross' : If(['bird','flies well'])
}

برای انجام استنتاج معکوس، ما کلاس `Knowledgebase` را تعریف خواهیم کرد. این کلاس شامل موارد زیر خواهد بود:
* `حافظه` کاری - یک دیکشنری که ویژگی‌ها را به مقادیر نگاشت می‌کند
* `قوانین` پایگاه دانش در قالبی که در بالا تعریف شده است

دو متد اصلی عبارتند از:
* `get` برای به دست آوردن مقدار یک ویژگی، با انجام استنتاج در صورت لزوم. به عنوان مثال، `get('color')` مقدار یک اسلات رنگ را دریافت می‌کند (در صورت نیاز سؤال می‌پرسد و مقدار را برای استفاده‌های بعدی در حافظه کاری ذخیره می‌کند). اگر ما `get('color:blue')` را بپرسیم، ابتدا رنگ را می‌پرسد و سپس مقدار `y`/`n` را بسته به رنگ بازمی‌گرداند.
* `eval` استنتاج واقعی را انجام می‌دهد، یعنی درخت AND/OR را پیمایش می‌کند، اهداف فرعی را ارزیابی می‌کند و غیره.


In [3]:
class KnowledgeBase():
    def __init__(self,rules):
        self.rules = rules
        self.memory = {}
        
    def get(self,name):
        if ':' in name:
            k,v = name.split(':')
            vv = self.get(k)
            return 'y' if v==vv else 'n'
        if name in self.memory.keys():
            return self.memory[name]
        for fld in self.rules.keys():
            if fld==name or fld.startswith(name+":"):
                # print(" + proving {}".format(fld))
                value = 'y' if fld==name else fld.split(':')[1]
                res = self.eval(self.rules[fld],field=name)
                if res!='y' and res!='n' and value=='y':
                    self.memory[name] = res
                    return res
                if res=='y':
                    self.memory[name] = value
                    return value
        # field is not found, using default
        res = self.eval(self.rules['default'],field=name)
        self.memory[name]=res
        return res
                
    def eval(self,expr,field=None):
        # print(" + eval {}".format(expr))
        if isinstance(expr,Ask):
            print(field)
            return expr.ask()
        elif isinstance(expr,If):
            return self.eval(expr.x)
        elif isinstance(expr,AND) or isinstance(expr,list):
            expr = expr.x if isinstance(expr,AND) else expr
            for x in expr:
                if self.eval(x)=='n':
                    return 'n'
            return 'y'
        elif isinstance(expr,OR):
            for x in expr.x:
                if self.eval(x)=='y':
                    return 'y'
            return 'n'
        elif isinstance(expr,str):
            return self.get(expr)
        else:
            print("Unknown expr: {}".format(expr))

حالا بیایید پایگاه دانش حیوانات خود را تعریف کنیم و مشاوره را انجام دهیم. توجه داشته باشید که این تماس از شما سوالاتی خواهد پرسید. شما می‌توانید با تایپ کردن `y`/`n` برای سوالات بله-خیر پاسخ دهید، یا برای سوالات با پاسخ‌های چند گزینه‌ای طولانی‌تر، عددی (۰..N) مشخص کنید.


In [4]:
kb = KnowledgeBase(rules)
kb.get('animal')

hair
y/n
sharp teeth
y/n
claws
y/n
forward-looking eyes
y/n
color
0. red-brown
1. black and white
2. other
has hooves
y/n
long neck
y/n
long legs
y/n
pattern
0. dark stripes
1. dark spots


'giraffe'

## استفاده از PyKnow برای استنتاج پیشرو

در مثال بعدی، سعی می‌کنیم استنتاج پیشرو را با استفاده از یکی از کتابخانه‌های نمایش دانش، [PyKnow](https://github.com/buguroo/pyknow/) پیاده‌سازی کنیم. **PyKnow** یک کتابخانه برای ایجاد سیستم‌های استنتاج پیشرو در پایتون است که به گونه‌ای طراحی شده تا شبیه به سیستم کلاسیک قدیمی [CLIPS](http://www.clipsrules.net/index.html) باشد.

ما می‌توانستیم استنتاج پیشرو را خودمان نیز بدون مشکلات زیادی پیاده‌سازی کنیم، اما پیاده‌سازی‌های ساده معمولاً خیلی کارآمد نیستند. برای تطبیق قوانین به شکل مؤثرتر، از یک الگوریتم خاص به نام [Rete](https://en.wikipedia.org/wiki/Rete_algorithm) استفاده می‌شود.


In [5]:
import sys
!{sys.executable} -m pip install git+https://github.com/buguroo/pyknow/

  Cloning https://github.com/buguroo/pyknow/ to /tmp/pip-req-build-3cqeulyl
  Running command git clone --filter=blob:none --quiet https://github.com/buguroo/pyknow/ /tmp/pip-req-build-3cqeulyl
  Resolved https://github.com/buguroo/pyknow/ to commit 48818336f2e9a126f1964f2d8dc22d37ff800fe8
  Preparing metadata (setup.py) ... done
  Using cached frozendict-1.2.tar.gz (2.6 kB)
  Preparing metadata (setup.py) ... done
  Using cached schema-0.6.7-py2.py3-none-any.whl (14 kB)
  Created wheel for pyknow: filename=pyknow-1.7.0-py3-none-any.whl size=34228 sha256=b7de5b09292c4007667c72f69b98d5a1b5f7324ff15f9dd8e077c3d5f7aade42
  Stored in directory: /tmp/pip-ephem-wheel-cache-k7jpave7/wheels/81/1a/d3/f6c15dbe1955598a37755215f2a10449e7418500d7bd4b9508
  Created wheel for frozendict: filename=frozendict-1.2-py3-none-any.whl size=3148 sha256=2863d55c240d2409cddf05ccfe600591f8478681549fc97555c47c90dc6bb160
  Stored in directory: /home/rg/.cache/pip/wheels/49/ac/f8/cb8120244e710bdb479c86198b03c7b08c

In [13]:
from pyknow import *
#import pyknow

ما سیستم خود را به عنوان یک کلاس که از `KnowledgeEngine` ارث‌بری می‌کند تعریف خواهیم کرد. هر قانون توسط یک تابع جداگانه با حاشیه‌نویسی `@Rule` تعریف می‌شود که مشخص می‌کند چه زمانی قانون باید اجرا شود. داخل قانون، می‌توانیم با استفاده از تابع `declare` حقایق جدیدی اضافه کنیم و افزودن این حقایق باعث می‌شود که قوانین بیشتری توسط موتور استنتاج پیشرو فراخوانی شوند.


In [14]:
class Animals(KnowledgeEngine):
    @Rule(OR(
           AND(Fact('sharp teeth'),Fact('claws'),Fact('forward looking eyes')),
           Fact('eats meat')))
    def cornivor(self):
        self.declare(Fact('carnivor'))
        
    @Rule(OR(Fact('hair'),Fact('gives milk')))
    def mammal(self):
        self.declare(Fact('mammal'))

    @Rule(Fact('mammal'),
          OR(Fact('has hooves'),Fact('chews cud')))
    def hooves(self):
        self.declare('ungulate')
        
    @Rule(OR(Fact('feathers'),AND(Fact('flies'),Fact('lays eggs'))))
    def bird(self):
        self.declare('bird')
        
    @Rule(Fact('mammal'),Fact('carnivor'),
          Fact(color='red-brown'),
          Fact(pattern='dark spots'))
    def monkey(self):
        self.declare(Fact(animal='monkey'))

    @Rule(Fact('mammal'),Fact('carnivor'),
          Fact(color='red-brown'),
          Fact(pattern='dark stripes'))
    def tiger(self):
        self.declare(Fact(animal='tiger'))

    @Rule(Fact('ungulate'),
          Fact('long neck'),
          Fact('long legs'),
          Fact(pattern='dark spots'))
    def giraffe(self):
        self.declare(Fact(animal='giraffe'))

    @Rule(Fact('ungulate'),
          Fact(pattern='dark stripes'))
    def zebra(self):
        self.declare(Fact(animal='zebra'))

    @Rule(Fact('bird'),
          Fact('long neck'),
          Fact('cannot fly'),
          Fact(color='black and white'))
    def straus(self):
        self.declare(Fact(animal='ostrich'))

    @Rule(Fact('bird'),
          Fact('swims'),
          Fact('cannot fly'),
          Fact(color='black and white'))
    def pinguin(self):
        self.declare(Fact(animal='pinguin'))

    @Rule(Fact('bird'),
          Fact('flies well'))
    def albatros(self):
        self.declare(Fact(animal='albatross'))
        
    @Rule(Fact(animal=MATCH.a))
    def print_result(self,a):
          print('Animal is {}'.format(a))
                    
    def factz(self,l):
        for x in l:
            self.declare(x)

هنگامی که یک پایگاه دانش تعریف کردیم، حافظه کاری خود را با برخی حقایق اولیه پر می‌کنیم و سپس متد `run()` را فراخوانی می‌کنیم تا استنتاج انجام شود. می‌توانید مشاهده کنید که در نتیجه، حقایق استنتاج‌شده جدید به حافظه کاری اضافه می‌شوند، از جمله حقیقت نهایی درباره حیوان (اگر تمام حقایق اولیه را به درستی تنظیم کرده باشیم).


In [15]:
ex1 = Animals()
ex1.reset()
ex1.factz([
    Fact(color='red-brown'),
    Fact(pattern='dark stripes'),
    Fact('sharp teeth'),
    Fact('claws'),
    Fact('forward looking eyes'),
    Fact('gives milk')])
ex1.run()
ex1.facts

Animal is tiger


FactList([(0, InitialFact()),
          (1, Fact(color='red-brown')),
          (2, Fact(pattern='dark stripes')),
          (3, Fact('sharp teeth')),
          (4, Fact('claws')),
          (5, Fact('forward looking eyes')),
          (6, Fact('gives milk')),
          (7, Fact('mammal')),
          (8, Fact('carnivor')),
          (9, Fact(animal='tiger'))])


---

**سلب مسئولیت**:  
این سند با استفاده از سرویس ترجمه هوش مصنوعی [Co-op Translator](https://github.com/Azure/co-op-translator) ترجمه شده است. در حالی که ما تلاش می‌کنیم دقت را حفظ کنیم، لطفاً توجه داشته باشید که ترجمه‌های خودکار ممکن است حاوی خطاها یا نادرستی‌هایی باشند. سند اصلی به زبان اصلی آن باید به عنوان منبع معتبر در نظر گرفته شود. برای اطلاعات حساس، ترجمه حرفه‌ای انسانی توصیه می‌شود. ما هیچ مسئولیتی در قبال سوءتفاهم‌ها یا تفسیرهای نادرست ناشی از استفاده از این ترجمه نداریم.
